# ML-2: Trees, Model Interrogation and Bayesian Workflow
# Homework 2: Rossman Kaggle: Forecasting Sales
# Part 4 : Modelling with embeddings!
**ML-2 Cohort 1** <br>
**Instructor: Dr. Rahul Dave**<br>
**Max Score: 100** <br>

In [13]:
#importing libraries
import numpy as np
import scipy.stats
import scipy.special
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import cm
import pandas as pd
from sklearn.pipeline import make_pipeline, make_union, Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import ParameterGrid, train_test_split
from keras.models import Sequential
from keras.models import Model as KerasModel
from keras.layers import Input, Dense, Activation, Reshape
from keras.layers import Concatenate
from keras.layers.embeddings import Embedding
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
import pickle
import csv
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

We will repeat the first initial steps again from Part 2

Lets import the feature_train_data.pickle file and set X,y values from the pickle file

In [2]:
#your code here
f = open('feature_train_data.pickle', 'rb') 
(X, y) = pickle.load(f)

In [3]:
# we will split the train_ratio and 90% and 10% and set the train_size
train_ratio = 0.9
num_records = len(X)
train_size = int(train_ratio * num_records)

In [4]:
#lets look at our data
X[1], y[1]

(array([   0, 1058,    1,    0,    0,    0,    0,    1]), 4491)

The next set of inputs is following: Write the code for this yourself

1. Do you want to one hot encode the data?
2. Do you want to provide embeddings as input - this will be set to True for models with entity embeddings
3. Do you want to save the emmbeddings? - again set to true if you want to entity embeddings
4. if 3 is set to true, we want to save them to a embeddings.pickle


In [5]:
#your code here
one_hot_as_input = False #one_hot is set to False
embeddings_as_input = True #embeddings later on needs to set to true for Part 3
save_embeddings = True
saved_embeddings_fname = "embeddings.pickle"  # set save_embeddings to True to create this file

## Now lets work with Models with Entity embedding!!

Now that you have saved embeddings - push this into the other models as an input with X. 

How will we do this? 

We need to update our X values. 

1. We will define a function called embed_features, which will combine the embedding with X. 
2. Call this function and update it with the inital X values taken from the pickle file - features_train_data
3. Then split you data, X_emb - into Xtrain and X_Val, y_train and y_val remain the same
4. Sample the data

In [9]:
#call our saved embeddings from part 3

saved_embeddings_fname = '/content/embeddings.pickle'

In [10]:
#combining embedding
def embed_features(X, saved_embeddings_fname):
    f_embeddings = open(saved_embeddings_fname, "rb")
    embeddings = pickle.load(f_embeddings)

    index_embedding_mapping = {1: 0, 2: 1, 4: 2, 5: 3, 6: 4, 7: 5}
    X_embedded = []

    (num_records, num_features) = X.shape
    for record in X:
        embedded_features = []
        for i, feat in enumerate(record):
            feat = int(feat)
            if i not in index_embedding_mapping.keys():
                embedded_features += [feat]
            else:
                embedding_index = index_embedding_mapping[i]
                embedded_features += embeddings[embedding_index][feat].tolist()

        X_embedded.append(embedded_features)

    return np.array(X_embedded)

**Explain what is happening the function above**

your answer here

### Embedding with X - input

In [11]:
#check if embedding is needed, if yes call embed_features - with X and the embeddings passed to it - call this new X as X_emb
if embeddings_as_input:
   X_emb = embed_features(X, saved_embeddings_fname)


Split the train and validation based on train size and on the new X_emb values from the previous code

In [14]:
#update the X_train and X_val
#your code here 

X_train, X_val, y_train, y_val = train_test_split(X_emb, y, train_size= train_size, random_state=42)

In [15]:
def sample(X, y, n):
    '''random samples'''
    num_row = X.shape[0]
    indices = np.random.randint(num_row, size=n)
    return X[indices, :], y[indices]

In [16]:
X_train, y_train = sample(X_train, y_train, 200000)  # Simulate data sparsity
print("Number of samples used for training: " + str(y_train.shape[0]))

Number of samples used for training: 200000


## Add the embeddings into the Models and check their MAPE!

All the models defined here will have the same parameters as the ones defined in Part 2!

In [17]:
#define MAPE Function here
#your code here

def MAPE(Y_actual,Y_Predicted):
  return np.mean(np.abs((Y_actual - Y_Predicted) / Y_actual)) * 100

### Model 1: Random Forests

In [ ]:
#your code here

rf_reg = RandomForestRegressor(n_estimators = 200, max_depth = 35, min_samples_split = 2, min_samples_leaf = 1)
rf_reg.fit(X_train, y_train)

In [19]:
rf_train_pred = rf_reg.predict(X_train)
rf_val_pred = rf_reg.predict(X_val)

In [20]:
print("The mean absolute percentage error on training data is ", np.round(MAPE(y_train, rf_train_pred), 3))

The mean absolute percentage error on training data is  2.847


In [21]:
print("The mean absolute percentage error on validation data is ", np.round(MAPE(y_val, rf_val_pred), 3))

The mean absolute percentage error on validation data is  9.076


### Model 2: Boosting

In [22]:
#your code here
DM_train = xgb.DMatrix(data = X_train, 
                       label = y_train)


In [23]:
param  = {'nthread' : -1, 
           'max_depth': 7, 
           'eta': 0.02,
           'silent': 1,
           'objective': 'reg:linear',
           'colsample_bytree': 0.7,
           'subsample': 0.7}

num_round = 3000

In [24]:
bst = xgb.train(param, DM_train, num_round)

In [25]:
feature_Xtr = xgb.DMatrix(X_train)
feature_Xval = xgb.DMatrix(X_val)

In [26]:
xgb_y_pred_train = bst.predict(feature_Xtr)
xgb_y_pred_val = bst.predict(feature_Xval)

In [27]:
xgb_mape_train= MAPE(y_train, xgb_y_pred_train)
xgb_mape_val= MAPE(y_val, xgb_y_pred_val)

In [28]:
print("The mean absolute percentage error on training data is ", np.round(xgb_mape_train, 3))

The mean absolute percentage error on training data is  5.99


In [29]:
print("The mean absolute percentage error on validation data is ", np.round(xgb_mape_val, 3))

The mean absolute percentage error on validation data is  7.183


## Final Commments!

Apart from how long this homework was, lets make some other final comments

**Question 1: Did you models with Entity Embeddings perfom better?**

No, it did not perform better. 


**Question 2: Now that you have completed this homework, lets answer the main purpose of the homework - How do you think entity embeddings improved the MAPE score. To show this do a similar table like the one did in Paper**

*your answer here*


**Question 3: Add a table here to show the similar to the one done in paper - to show the different MAPE values with and without embeddings**

*your answer here*